In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();

 Activating environment at `C:\Users\Administrator\Documents\GitHub\16-745-hw1\Project.toml`


In [2]:
using Random, LinearAlgebra

In [3]:
struct QPData
    P::Matrix{Float64}
    q::Vector{Float64}
    A::Matrix{Float64}
    b::Vector{Float64}
    C::Matrix{Float64}
    d::Vector{Float64}
end

In [4]:
function QPData(n::Int, m::Int, p::Int)
    QPData(zeros(n,n), zeros(n), zeros(m,n), zeros(m), zeros(p,n), zeros(p))
end

QPData

In [5]:
num_eq(qp::QPData) = length(qp.b)
num_ineq(qp::QPData) = length(qp.d)

num_ineq (generic function with 1 method)

In [6]:
Base.size(qp::QPData) = (length(qp.q), num_eq(qp), num_ineq(qp))

In [7]:
objective(qp::QPData, x) = 0.5 * x'qp.P*x + qp.q'x
ceq(qp::QPData, x) = qp.A * x - qp.b
cin(qp::QPData, x) = qp.C * x - qp.d

cin (generic function with 1 method)

In [8]:
function primal_residual(qp::QPData, x, λ, μ)
    qp.P*x + qp.q + qp.A'λ + qp.C'μ
end

primal_residual (generic function with 1 method)

In [9]:
function dual_residual(qp::QPData, x, λ, μ)
    g = ceq(qp, x)
    h = cin(qp, x)
    return [g; max.(0, h)]
end

dual_residual (generic function with 1 method)

In [10]:
function complimentarity(qp::QPData, x, λ, μ)
    return [min.(0, μ); μ .* cin(qp, x)]
end

complimentarity (generic function with 1 method)